In [1]:
import requests
from bs4 import BeautifulSoup
import random
import csv

# with open('proxies.txt', 'r') as f:
#    proxies = f.read().splitlines()

# proxy = random.choice(proxies)

header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
          'referer':'https://www.zillow.com/homes/Missoula,-MT_rb/'}

#response = requests.get('https://www.zillow.com/portland-or/', headers=header, proxies={"http": proxy})
response = requests.get('https://www.zillow.com/la-jolla-san-diego-ca/', headers=header)

properties = []

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    search_results = soup.find(id="grid-search-results")
    if search_results:  
        homecards = search_results.find_all("li")
        for card in homecards:
            if card.find("address", {"data-test": "property-card-addr"}):
                more_info = card.find("div", class_="property-card-data")
                info = more_info.find_all("li")
                data = {
                    "address": card.find("address", {"data-test": "property-card-addr"}).text.strip(),
                    "broker": more_info.find("div").text.strip(),
                    "price": card.find("span", {"data-test": "property-card-price"}).text.strip(),
                    "beds": info[0].text.strip(),
                    "bathrooms": info[1].text.strip(),
                    "sqft": info[2].text.strip(),
                    "url": card.find("a", {"data-test": "property-card-link"})["href"]
                }
                properties.append(data)
                print(data)

csv_header = ["Address", "Broker", "Price", "Beds", "Bathrooms", "Square Footage", "URL"]

with open("zillow.csv", "w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=csv_header)
    writer.writeheader()
    for property in properties:
        writer.writerow({
            "Address": property["address"],
            "Broker": property["broker"],
            "Price": property["price"],
            "Beds": property["beds"],
            "Bathrooms": property["bathrooms"],
            "Square Footage": property["sqft"],
            "URL": property["url"]
        })

{'address': '1900 Spindrift Dr, La Jolla, CA 92037', 'broker': 'COMPASS', 'price': '$108,000,000', 'beds': '10 bds', 'bathrooms': '17 ba', 'sqft': '12,981 sqft', 'url': 'https://www.zillow.com/homedetails/1900-Spindrift-Dr-La-Jolla-CA-92037/16839110_zpid/'}
{'address': '801 La Jolla Rancho Rd, La Jolla, CA 92037', 'broker': 'BERKSHIRE HATHAWAY HOMESERVICES CALIFORNIA PROPERTIES', 'price': '$3,995,000', 'beds': '3 bds', 'bathrooms': '3 ba', 'sqft': '2,890 sqft', 'url': 'https://www.zillow.com/homedetails/801-La-Jolla-Rancho-Rd-La-Jolla-CA-92037/16855358_zpid/'}
{'address': '6283 La Jolla Scenic Dr S, La Jolla, CA 92037', 'broker': 'EXP REALTY OF CALIFORNIA, INC.', 'price': '$22,500,000', 'beds': '7 bds', 'bathrooms': '10 ba', 'sqft': '12,842 sqft', 'url': 'https://www.zillow.com/homedetails/6283-La-Jolla-Scenic-Dr-S-La-Jolla-CA-92037/16852003_zpid/'}
{'address': '6653 Neptune Pl, La Jolla, CA 92037', 'broker': 'PACASO INC.', 'price': '$1,300,000', 'beds': '4 bds', 'bathrooms': '5 ba', '